# villager와 special npc를 합칩니다.

## 목표

- [ ] villager 필드 정리하고 special npc 합치기
- [ ] tier, ranking 합치기
  

In [80]:
import itertools

import numpy as np
import pandas as pd

In [58]:
chars = pd.read_excel('./datasets/villagers.xlsx')
npcs = pd.read_excel('./datasets/special npcs.xlsx')
pops = pd.read_excel('./datasets/popularity.xlsx')
trans = pd.read_csv('./datasets/koname.txt', delimiter='\t')

In [59]:
trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      463 non-null    object
 1   en      463 non-null    object
 2   ko      463 non-null    object
dtypes: object(3)
memory usage: 11.0+ KB


생각해보니 번역 파일에 빠진게 있으니 하는김에 채워넣어야 할 것 같습니다.

In [60]:
trans.drop('id', axis=1, inplace=True)
trans.head()

,en,ko
0,Cyrus,리포
1,Copper,경찰관
2,Wilbur,로드리
3,DJ KK,DJ K.K.
4,Booker,경찰관


In [61]:
missing = {
    'Shino': '요비',
    'Sasha': '미첼',
    'Ione': '스피카',
    'Cephalobot': '기가',
    'Tiansheng': '제천',
    'Petri': '리카',
    'Quinn': '시온',
    'Marlo': '보스터',
    'Azalea': '페튜니아',
    'Faith': '마치',
    'Chabwick': '펭구',
    'WartJr': '샘',
    'Rio': '데자네',
    'Buck(Brows)': '바야시코프',
    'Renee': '뿔님이',
    'Roswell': '삐로코',
    'Zoe': '소면',
    'OHare': '산토스',
    'Crackle(Spork)': '포크',
    'Ace': '페더',
    'Frett': '샹펜',
}

en, ko = zip(*missing.items())
# print(en, ko)
amend = pd.DataFrame({'en': en, 'ko': ko})
# amend.head()

trans = pd.concat([trans, amend], axis=0)
trans.reset_index()


,index,en,ko
0,0,Cyrus,리포
1,1,Copper,경찰관
2,2,Wilbur,로드리
3,3,DJ KK,DJ K.K.
4,4,Booker,경찰관
...,...,...,...
479,16,Zoe,소면
480,17,OHare,산토스
481,18,Crackle(Spork),포크
482,19,Ace,페더


In [62]:
trans.reset_index(inplace=True)

In [63]:
trans = trans.sort_values('en').reset_index()
trans.head()

,level_0,index,en,ko
0,64,64,???,???
1,482,19,Ace,페더
2,151,151,Admiral,일섭
3,415,415,Agent S,2호
4,83,83,Agnes,아그네스


In [64]:
trans.drop(['level_0', 'index'], axis=1, inplace=True)
trans.head()

,en,ko
0,???,???
1,Ace,페더
2,Admiral,일섭
3,Agent S,2호
4,Agnes,아그네스


In [65]:
trans.to_excel('./datasets/trans.xlsx')

## 마을사람 필드 정리

뺄 필드: Default Clothing	Wallpaper	Flooring	Furniture List	Furniture Name List	Filename	Unique Entry ID


In [67]:
todrop = [
    'Default Clothing',
    'Wallpaper',
    'Flooring',
    'Furniture List',
    'Furniture Name List',
    'Filename',
    'Unique Entry ID'
]
chars.drop(todrop, axis=1, inplace=True)
chars.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 391 entries, 0 to 390
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       391 non-null    int64 
 1   Name             391 non-null    object
 2   Icon Image       391 non-null    object
 3   Photo Image      391 non-null    object
 4   House Image      391 non-null    object
 5   Species          391 non-null    object
 6   Gender           391 non-null    object
 7   Personality      391 non-null    object
 8   Subtype          391 non-null    object
 9   Hobby            391 non-null    object
 10  Birthday         391 non-null    object
 11  Catchphrase      391 non-null    object
 12  Favorite Song    391 non-null    object
 13  Favorite Saying  391 non-null    object
 14  Style 1          391 non-null    object
 15  Style 2          391 non-null    object
 16  Color 1          391 non-null    object
 17  Color 2          391 non-null    ob

In [72]:
chars.drop('Unnamed: 0', axis=1, inplace=True)
chars.head()

,Name,Icon Image,Photo Image,House Image,Species,Gender,Personality,Subtype,Hobby,Birthday,Catchphrase,Favorite Song,Favorite Saying,Style 1,Style 2,Color 1,Color 2,Korean Name
0,Admiral,https://acnhcdn.com/latest/NpcIcon/brd06.png,https://acnhcdn.com/latest/NpcBromide/NpcNmlBr...,https://acnhcdn.com/drivesync/render/houses/br...,Bird,Male,Cranky,A,Nature,1/27,aye aye,Steep Hill,Only quitters give up.,Cool,Cool,Black,Blue,일섭
1,Agent S,https://acnhcdn.com/latest/NpcIcon/squ05.png,https://acnhcdn.com/latest/NpcBromide/NpcNmlSq...,https://acnhcdn.com/drivesync/render/houses/sq...,Squirrel,Female,Peppy,B,Fitness,7/2,sidekick,Go K.K. Rider,You gotta put the pedal to the metal!,Active,Simple,Blue,Black,2호
2,Agnes,https://acnhcdn.com/latest/NpcIcon/pig17.png,https://acnhcdn.com/latest/NpcBromide/NpcNmlPi...,https://acnhcdn.com/drivesync/render/houses/pi...,Pig,Female,Big Sister,A,Play,4/21,snuffle,K.K. House,You reap what you sow.,Simple,Elegant,Pink,White,아그네스
3,Al,https://acnhcdn.com/latest/NpcIcon/gor08.png,https://acnhcdn.com/latest/NpcBromide/NpcNmlGo...,https://acnhcdn.com/drivesync/render/houses/go...,Gorilla,Male,Lazy,B,Fitness,10/18,ayyyeee,Go K.K. Rider,Life is a buffet. Always go back for seconds.,Active,Active,Red,White,우락
4,Alfonso,https://acnhcdn.com/latest/NpcIcon/crd00.png,https://acnhcdn.com/latest/NpcBromide/NpcNmlCr...,https://acnhcdn.com/drivesync/render/houses/cr...,Alligator,Male,Lazy,B,Play,6/9,it'sa me,Forest Life,Slow and steady wins the race.,Simple,Simple,Red,Blue,알베르트


```js
const charactersMock = {
  admiral: {
    id: "admiral",
    special: false,
    name_en: "Admiral",
    name_ko: "일섭",
    image_icon: "https://acnhcdn.com/latest/NpcIcon/brd06.png",
    image_photo: "https://acnhcdn.com/latest/NpcBromide/NpcNmlBrd06.png",
    image_house:
      "https://acnhcdn.com/drivesync/render/houses/brd06_39_Admiral.png",
    species: "Bird",
    gender: "Male",
    gender_asia: "Male",
    personality: "Cranky",
    personality_subtype: "A",
    hobby: "Nature",
    birthday: "01-27",
    birthday_month: 1,
    birthday_day: 27,
    catchphrase: "aye aye",
    favorite_song: "Steep Hill",
    favorite_saying: "Only quitters give up.",
    styles: ["Cool"],
    colors: ["Black", "Blue"],
  },
  cyrus: {
    id: "cyrus",
    special: true,
    name_en: "Cyrus",
    name_ko: "리포",
    image_icon: "https://acnhcdn.com/latest/NpcIcon/alp.png",
    image_photo: "https://acnhcdn.com/latest/NpcPoster/NpcSpAlp.png",
    gender: "Male",
    gender_asia: "Male",
    birthday: "01-26",
    birthday_month: 1,
    birthday_day: 26,
  },
};
```

In [75]:
torename = {
    'Name': 'name_en',
    'Korean Name': 'name_ko',
    'Icon Image': 'image_icon',
    'Photo Image': 'image_photo',
    'House Image': 'image_house',
    'Species': 'species',
    'Gender': 'gender',
    'Gender (Asia)': 'gender_asia',
    'Personality': 'personality',
    'Subtype': 'personality_subtype',
    'Hobby': 'hobby',
    'Birthday': 'birthday',
    'Catchphrase': 'catchphrase',
    'Favorite Song': 'favorite_song',
    'Favorite Saying': 'favorite_saying',
}

chars.rename(columns=torename, inplace=True)
chars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 391 entries, 0 to 390
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   name_en              391 non-null    object
 1   image_icon           391 non-null    object
 2   image_photo          391 non-null    object
 3   image_house          391 non-null    object
 4   species              391 non-null    object
 5   gender               391 non-null    object
 6   personality          391 non-null    object
 7   personality_subtype  391 non-null    object
 8   hobby                391 non-null    object
 9   birthday             391 non-null    object
 10  catchphrase          391 non-null    object
 11  favorite_song        391 non-null    object
 12  favorite_saying      391 non-null    object
 13  Style 1              391 non-null    object
 14  Style 2              391 non-null    object
 15  Color 1              391 non-null    object
 16  Color 2 

### 마을 사람 후처리

특정 필드를 만들어 넣습니다.

In [78]:
chars['gender_asia'] = chars['gender'].copy()
chars.head()


,name_en,image_icon,image_photo,image_house,species,gender,personality,personality_subtype,hobby,birthday,catchphrase,favorite_song,favorite_saying,Style 1,Style 2,Color 1,Color 2,name_ko,gender_asia
0,Admiral,https://acnhcdn.com/latest/NpcIcon/brd06.png,https://acnhcdn.com/latest/NpcBromide/NpcNmlBr...,https://acnhcdn.com/drivesync/render/houses/br...,Bird,Male,Cranky,A,Nature,1/27,aye aye,Steep Hill,Only quitters give up.,Cool,Cool,Black,Blue,일섭,Male
1,Agent S,https://acnhcdn.com/latest/NpcIcon/squ05.png,https://acnhcdn.com/latest/NpcBromide/NpcNmlSq...,https://acnhcdn.com/drivesync/render/houses/sq...,Squirrel,Female,Peppy,B,Fitness,7/2,sidekick,Go K.K. Rider,You gotta put the pedal to the metal!,Active,Simple,Blue,Black,2호,Female
2,Agnes,https://acnhcdn.com/latest/NpcIcon/pig17.png,https://acnhcdn.com/latest/NpcBromide/NpcNmlPi...,https://acnhcdn.com/drivesync/render/houses/pi...,Pig,Female,Big Sister,A,Play,4/21,snuffle,K.K. House,You reap what you sow.,Simple,Elegant,Pink,White,아그네스,Female
3,Al,https://acnhcdn.com/latest/NpcIcon/gor08.png,https://acnhcdn.com/latest/NpcBromide/NpcNmlGo...,https://acnhcdn.com/drivesync/render/houses/go...,Gorilla,Male,Lazy,B,Fitness,10/18,ayyyeee,Go K.K. Rider,Life is a buffet. Always go back for seconds.,Active,Active,Red,White,우락,Male
4,Alfonso,https://acnhcdn.com/latest/NpcIcon/crd00.png,https://acnhcdn.com/latest/NpcBromide/NpcNmlCr...,https://acnhcdn.com/drivesync/render/houses/cr...,Alligator,Male,Lazy,B,Play,6/9,it'sa me,Forest Life,Slow and steady wins the race.,Simple,Simple,Red,Blue,알베르트,Male


In [82]:
chars['special'] = list(itertools.repeat(False, len(chars)))
chars['special'].head()

0    False
1    False
2    False
3    False
4    False
Name: special, dtype: bool

In [91]:
chars['styles'] = chars.apply(
    lambda row: list(set([row['Style 1'], row['Style 2']])),
    axis=1
)
chars['styles'].head()

0               [Cool]
1     [Simple, Active]
2    [Simple, Elegant]
3             [Active]
4             [Simple]
Name: styles, dtype: object

In [92]:
chars['colors'] = chars.apply(
    lambda row: list(set([row['Color 1'], row['Color 2']])),
    axis=1
)
chars['colors'].head()

0    [Black, Blue]
1    [Black, Blue]
2    [White, Pink]
3     [White, Red]
4      [Red, Blue]
Name: colors, dtype: object

In [95]:
chars.drop(['Style 1', 'Style 2', 'Color 1', 'Color 2'], axis=1, inplace=True)
chars.head()

,name_en,image_icon,image_photo,image_house,species,gender,personality,personality_subtype,hobby,birthday,catchphrase,favorite_song,favorite_saying,name_ko,gender_asia,special,styles,colors
0,Admiral,https://acnhcdn.com/latest/NpcIcon/brd06.png,https://acnhcdn.com/latest/NpcBromide/NpcNmlBr...,https://acnhcdn.com/drivesync/render/houses/br...,Bird,Male,Cranky,A,Nature,1/27,aye aye,Steep Hill,Only quitters give up.,일섭,Male,False,[Cool],"[Black, Blue]"
1,Agent S,https://acnhcdn.com/latest/NpcIcon/squ05.png,https://acnhcdn.com/latest/NpcBromide/NpcNmlSq...,https://acnhcdn.com/drivesync/render/houses/sq...,Squirrel,Female,Peppy,B,Fitness,7/2,sidekick,Go K.K. Rider,You gotta put the pedal to the metal!,2호,Female,False,"[Simple, Active]","[Black, Blue]"
2,Agnes,https://acnhcdn.com/latest/NpcIcon/pig17.png,https://acnhcdn.com/latest/NpcBromide/NpcNmlPi...,https://acnhcdn.com/drivesync/render/houses/pi...,Pig,Female,Big Sister,A,Play,4/21,snuffle,K.K. House,You reap what you sow.,아그네스,Female,False,"[Simple, Elegant]","[White, Pink]"
3,Al,https://acnhcdn.com/latest/NpcIcon/gor08.png,https://acnhcdn.com/latest/NpcBromide/NpcNmlGo...,https://acnhcdn.com/drivesync/render/houses/go...,Gorilla,Male,Lazy,B,Fitness,10/18,ayyyeee,Go K.K. Rider,Life is a buffet. Always go back for seconds.,우락,Male,False,[Active],"[White, Red]"
4,Alfonso,https://acnhcdn.com/latest/NpcIcon/crd00.png,https://acnhcdn.com/latest/NpcBromide/NpcNmlCr...,https://acnhcdn.com/drivesync/render/houses/cr...,Alligator,Male,Lazy,B,Play,6/9,it'sa me,Forest Life,Slow and steady wins the race.,알베르트,Male,False,[Simple],"[Red, Blue]"


## special npc 후처리 및 정리


In [97]:
npcs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       63 non-null     int64 
 1   Name             63 non-null     object
 2   Icon Image       22 non-null     object
 3   Photo Image      54 non-null     object
 4   Gender           63 non-null     object
 5   Gender (Asia)    63 non-null     object
 6   Version Added    29 non-null     object
 7   NPC ID           63 non-null     object
 8   Internal ID      63 non-null     int64 
 9   Birthday         63 non-null     object
 10  Unique Entry ID  63 non-null     object
 11  Korean Name      63 non-null     object
dtypes: int64(2), object(10)
memory usage: 6.0+ KB


In [98]:
npcs.drop(
    [
        'Unnamed: 0',
        'Version Added',
        'NPC ID',
        'Internal ID',
        'Unique Entry ID'
    ],
    axis=1,
    inplace=True
)
npcs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Name           63 non-null     object
 1   Icon Image     22 non-null     object
 2   Photo Image    54 non-null     object
 3   Gender         63 non-null     object
 4   Gender (Asia)  63 non-null     object
 5   Birthday       63 non-null     object
 6   Korean Name    63 non-null     object
dtypes: object(7)
memory usage: 3.6+ KB


In [100]:
npcs.rename(columns=torename, inplace=True)
npcs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name_en      63 non-null     object
 1   image_icon   22 non-null     object
 2   image_photo  54 non-null     object
 3   gender       63 non-null     object
 4   gender_asia  63 non-null     object
 5   birthday     63 non-null     object
 6   name_ko      63 non-null     object
dtypes: object(7)
memory usage: 3.6+ KB


필요한 칼럼은 `special` 하나 뿐인 것 같습니다.


In [105]:
npcs = npcs.assign(special=lambda v: True)

이제 `chars`와 합칩니다.


In [117]:
chars = pd.concat([chars, npcs], axis=0).reset_index()
chars.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 454 entries, 0 to 453
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   index                454 non-null    int64 
 1   name_en              454 non-null    object
 2   image_icon           413 non-null    object
 3   image_photo          445 non-null    object
 4   image_house          391 non-null    object
 5   species              391 non-null    object
 6   gender               454 non-null    object
 7   personality          391 non-null    object
 8   personality_subtype  391 non-null    object
 9   hobby                391 non-null    object
 10  birthday             454 non-null    object
 11  catchphrase          391 non-null    object
 12  favorite_song        391 non-null    object
 13  favorite_saying      391 non-null    object
 14  name_ko              454 non-null    object
 15  gender_asia          454 non-null    object
 16  special 

## `birthday` 숫자 필드 만들기

`birthday_month`, `birthday_day`


In [121]:
# birthday 칼럼을 mm-dd로 고칩니다.
def reformat_birthday(bdstr):
    m, d = map(int, bdstr.split('/'))
    return f'{m:02}-{d:02}'

# reformat_birthday('1/26')
chars['birthday'] = chars['birthday'].apply(reformat_birthday)
chars['birthday'].head()

0    01-27
1    07-02
2    04-21
3    10-18
4    06-09
Name: birthday, dtype: object

In [131]:
chars['birthday_month'] = chars['birthday'].apply(lambda v: int(v[:2]))
chars['birthday_day'] = chars['birthday'].apply(lambda v: int(v[-2:]))
chars.drop('index', axis=1, inplace=True)


AttributeError: 'NoneType' object has no attribute 'head'

In [137]:
chars[chars['name_en'].duplicated()]

,name_en,image_icon,image_photo,image_house,species,gender,personality,personality_subtype,hobby,birthday,catchphrase,favorite_song,favorite_saying,name_ko,gender_asia,special,styles,colors,birthday_month,birthday_day
409,Wisp,NaN,NaN,NaN,NaN,Male,NaN,NaN,NaN,02-26,NaN,NaN,NaN,깨빈,Male,True,NaN,NaN,2,26


Wisp 이란 녀석이 중복이고 어떤 녀석들은 아이콘 이미지가 없습니다.

~~이건 수작업으로 해야 할 것 같기 때문에 일단 익스포트합니다.~~


In [140]:
chars.drop(409, inplace=True)

In [144]:
chars = chars.sort_values('name_en').reset_index()

## 번역

영어 속성을 번역합니다.
